In [2]:
# Import python libraries
#
import pandas as pd
import numpy as np
import importlib
import pickle
import os

# Go one directory back, because all imports are done 
# relative to the root of the project.
#
if 'change_directory_to_root' not in globals():
    change_directory_to_root = True
    project_root = '..'
    os.chdir(project_root)

# Imports own modules.
#
import scripts.Visualization as Visualization
import scripts.ModelTrainer as ModelTrainer
import scripts.utils as utils
import scripts.Simulation_config as config
import scripts.Model as model


/home/molu/miniconda3/envs/xlstm/lib/python3.11/site-packages/torch/cuda/__init__.py:141: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11040). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at /opt/conda/conda-bld/pytorch_1704987288773/work/c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/home/molu/miniconda3/envs/xlstm'


In [2]:
# Run the whole simulation
#
importlib.reload(model)
importlib.reload(ModelTrainer)
ModelTrainer.ModelTrainer().run()


Do Data Preprocessing.



Processing model LSTM with load profile scripts/outputs/file_0.pkl
...
Processing model LSTM with load profile scripts/outputs/file_1.pkl
...

In [2]:
importlib.reload(utils)

all_train_histories = utils.Deserialize.get_training_histories('scripts/outputs/all_train_histories.pkl')
all_trained_models = utils.Deserialize.get_trained_models('scripts/outputs/all_trained_models.pth')


In [27]:
all_train_histories
train_losses

{('xLSTM',
  'scripts/outputs/file_0.pkl',
  Config_of_one_simulation(DoPretraining=True, ModelSize=2, Aggregation_Count='data/london_loadprofiles.pkl', NrOfComunities=20, TrainingHistory=4, InputSequenceLength=1, DoTransferLearning=True, UsedModels=('xLSTM', 'LSTM', 'Transformer', 'KNN', 'PersistencePrediction'))): 0.08432225400158264,
 ('xLSTM',
  'scripts/outputs/file_1.pkl',
  Config_of_one_simulation(DoPretraining=True, ModelSize=2, Aggregation_Count='data/london_loadprofiles.pkl', NrOfComunities=20, TrainingHistory=4, InputSequenceLength=1, DoTransferLearning=True, UsedModels=('xLSTM', 'LSTM', 'Transformer', 'KNN', 'PersistencePrediction'))): 0.09651134690757474,
 ('xLSTM',
  'scripts/outputs/file_2.pkl',
  Config_of_one_simulation(DoPretraining=True, ModelSize=2, Aggregation_Count='data/london_loadprofiles.pkl', NrOfComunities=20, TrainingHistory=4, InputSequenceLength=1, DoTransferLearning=True, UsedModels=('xLSTM', 'LSTM', 'Transformer', 'KNN', 'PersistencePrediction'))): 0.08

In [29]:
# Evaluate the results
#

# Get the loss out of the keras history object
train_losses, val_RMSE, val_sMAPE  = {}, {}, {}
for key, value in all_train_histories.items():
    train_losses[key] = np.sqrt(float(value['loss'][-1]))
    val_RMSE[key] = np.sqrt(float(value['test_loss'][-1]))
    val_sMAPE[key] = float(value['test_sMAPE'][-1])

def print_metric(metric):
    sorted_metric = dict(sorted(metric.items(), key=lambda item: item[1]))
    for test_id, value in sorted_metric.items():
        print(f"Abs.: {value:.4f}. With {test_id}")

print("Test-RMSE:")
print_metric(val_RMSE)
print("\n\nTrain-RMSE:")
print_metric(train_losses)


Test-RMSE:
Abs.: 0.3250. With ('xLSTM', 'scripts/outputs/file_5.pkl', Config_of_one_simulation(DoPretraining=True, ModelSize=2, Aggregation_Count='data/london_loadprofiles.pkl', NrOfComunities=20, TrainingHistory=4, InputSequenceLength=1, DoTransferLearning=True, UsedModels=('xLSTM', 'LSTM', 'Transformer', 'KNN', 'PersistencePrediction')))
Abs.: 0.3267. With ('LSTM', 'scripts/outputs/file_5.pkl', Config_of_one_simulation(DoPretraining=True, ModelSize=2, Aggregation_Count='data/london_loadprofiles.pkl', NrOfComunities=20, TrainingHistory=4, InputSequenceLength=1, DoTransferLearning=True, UsedModels=('xLSTM', 'LSTM', 'Transformer', 'KNN', 'PersistencePrediction')))
Abs.: 0.3316. With ('LSTM', 'scripts/outputs/file_16.pkl', Config_of_one_simulation(DoPretraining=True, ModelSize=2, Aggregation_Count='data/london_loadprofiles.pkl', NrOfComunities=20, TrainingHistory=4, InputSequenceLength=1, DoTransferLearning=True, UsedModels=('xLSTM', 'LSTM', 'Transformer', 'KNN', 'PersistencePrediction')

In [36]:
from collections import defaultdict

# Organize data by model_type
model_data_sMAPE = defaultdict(list)  # This kind of dict generates automatical an [] for each new key
for (model_type, _, _), value in val_sMAPE.items():
    model_data_sMAPE[model_type].append(value)

# Organize data by model_type
model_data_RMSE = defaultdict(list)  # This kind of dict generates automatical an [] for each new key
for (model_type, _, _), value in val_RMSE.items():
    model_data_RMSE[model_type].append(value)

# Calculate and print median and standard deviation for each model_type
for model_type, values in model_data_RMSE.items():
    print(f'Model: {model_type}')
    print(f'  Mean RMSE: {np.mean(values)}')
    print(f'  Mean sMAPE: {np.mean(model_data_sMAPE[model_type])}')
    print(f'  Standard Deviation RMSE: {np.std(values)}')
    print(f'  Standard Deviation sMAPE: {np.std(model_data_sMAPE[model_type])}\n')


Model: xLSTM
  Mean RMSE: 0.3905608792854397
  Mean sMAPE: 9.816956090927125
  Standard Deviation RMSE: 0.04223984653238611
  Standard Deviation sMAPE: 0.8746184693992828

Model: LSTM
  Mean RMSE: 0.3904177342581917
  Mean sMAPE: 9.902182245254517
  Standard Deviation RMSE: 0.047914850257516954
  Standard Deviation sMAPE: 0.9663037535751741

Model: Transformer
  Mean RMSE: 0.4096394978553678
  Mean sMAPE: 10.35506739616394
  Standard Deviation RMSE: 0.04346786695101477
  Standard Deviation sMAPE: 1.2793403847198674

Model: KNN
  Mean RMSE: 0.5156342183178657
  Mean sMAPE: 13.250053071975708
  Standard Deviation RMSE: 0.04970266045553766
  Standard Deviation sMAPE: 1.218492410221291

Model: PersistencePrediction
  Mean RMSE: 0.4529359654283819
  Mean sMAPE: 11.156612283401838
  Standard Deviation RMSE: 0.032337562790370866
  Standard Deviation sMAPE: 0.6747235520393585



In [34]:
import plotly.express as px

# Create a combined list of loss values and corresponding run names
combined_loss_data = []
for run_id, history in all_train_histories.items():
    combined_loss_data.extend([(run_id, epoch + 1, loss) for epoch, loss in enumerate(history['loss'])])

# Create a DataFrame from the combined data
import pandas as pd
df = pd.DataFrame(combined_loss_data, columns=['Run', 'Epoch', 'Loss'])

# Plot the loss of all items over the training epochs in one single plot
fig = px.line(df, x='Epoch', y='Loss', color='Run', labels={'Loss': 'Training Loss', 'Epoch': 'Epoch'})
fig.update_layout(title='Training Loss Over Epochs for Different Runs', showlegend=False)
fig.update_yaxes(range=[0, 1])
fig.show()


In [25]:
importlib.reload(Visualization)

# Evaluate the finetuned models
#
myModel = all_trained_models[('xLSTM', 'scripts/outputs/file_0.pkl', config.configs[0])]
test_profile = "scripts/outputs/file_0.pkl"
with open(test_profile, 'rb') as f:
    (X, Y, modelAdapter) = pickle.load(f)
plotlyApp = Visualization.PlotlyApp(X, Y, myModel, modelAdapter, None, 'UTC+00:00')
plotlyApp.run(myport=8055)


In [19]:
importlib.reload(model)

# Print the model parameter sizes
#
for model_type in ['xLSTM', 'LSTM', 'Transformer']:
    for model_size in ['SMALL', 'MEDIUM', 'LARGE']:

        m = model.Model(model_type, model_size)
        nr_of_parameters = m.get_nr_of_parameters(do_print=False)
        print(f"{model_type} - {model_size} has {nr_of_parameters} parameters.")


xLSTM - SMALL has 17671 parameters.
xLSTM - MEDIUM has 44151 parameters.
xLSTM - LARGE has 98763 parameters.
LSTM - SMALL has 17911 parameters.
LSTM - MEDIUM has 44911 parameters.
LSTM - LARGE has 106351 parameters.
Transformer - SMALL has 19651 parameters.
Transformer - MEDIUM has 47285 parameters.
Transformer - LARGE has 108981 parameters.


In [58]:
import torch
importlib.reload(model)

m = model.Model('Transformer', 'MEDIUM')
x = torch.zeros((7, 48, 20))
m.my_model(x).shape


torch.Size([7, 24, 1])